# Detect Deepfake Phishing Attacks

3/24/2025, Dave Sisk, https://github.com/davidcsisk, https://www.linkedin.com/in/davesisk-doctordatabase/

A number of AI-driven multi-model deepfake exploits have garnered much attention over the past year, including deepfaked audio/video phishing email of the company's CEO instructing employees to make a funds transfer, and even a CEO himself being deepfaked into transferring company funds to a fake corporate headquarters.  Let's examine how we can put together tools from the AI/ML/data science realm to detect this type of deepfake attack. 

By "multi-modal", I'm referring to attacks delivered as communications that might include text, audio, and video content. 

Based on results from a 2024 poll conducted by Deloitte, **24.9% of 2190 C-suite and executives say that their organization has experienced one or more deepfake attacks on financial and accounting targets**.

## Examples of real-world attacks from 2024 - present

Deepfaked CEO scam attempt: https://www.theguardian.com/technology/article/2024/may/10/ceo-wpp-deepfake-scam

Private video sharing phishing attack: https://incidentdatabase.ai/cite/965#r4845

Deepfaked Brad Pitt and the $850K romance scam:  https://incidentdatabase.ai/cite/901/

#### The proposed AI-driven Attack Detection Framework:
![CybersecurityAI-AnomalyDetection-Flowchart.jpg](.\CybersecurityAI-AnomalyDetection-Flowchart.jpg)

## Fabricated Example Attacks For Analysis:
The fabricated artifacts below were designed to appear realistic and relevant to our organization.

**If an audio message like the one below came in what appeared to be an official communication, how believable would it be?**

In [1]:
# Play the audio using the default OS audio player
wav_file = './deepfake_DonaldTrump-audio.wav'
!start {wav_file}

**If a video message similar to the one below was sent to all employees from what appeared to be an official source, how believable would it be?**

In [2]:
# Play the video using the default OS video player
video_file = './deepfake_ElonMusk_SocialSecurityPhishing.mp4'
!start {video_file}

I used inexpensive and readily-available AI tech to create both of these deepfakes.  If these examples are not believable enough, consider that it's simply a matter of using better models and tech to create deepfakes.  Overall, the obvious answer to the question at hand is this:  These are believable enough to potentially cause harm.

## Build library of known phishing attacks
In the proposed framework, this datastore of known attacks would be built over time by the reinforcement learning loop.

In [3]:
# Build the datastore of known phishing emails (30 samples)
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import warnings

# Suppress the FutureWarning from transformers
warnings.filterwarnings('ignore', category=FutureWarning, module='transformers.tokenization_utils_base')

# Load the CSV into a pandas dataframe
exploit_df = pd.read_csv('./deepfake_phishing_examples.csv')

# Ensure 'Subject' and 'Body' columns exist, create them if missing
if 'Subject' not in exploit_df.columns:
    exploit_df['Subject'] = ''
if 'Body' not in exploit_df.columns:
    exploit_df['Body'] = ''

# Initialize the sentence-transformers model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Combine Subject and Body into a single column for embedding, handling empty values
exploit_df['Subject'] = exploit_df['Subject'].fillna('')
exploit_df['Body'] = exploit_df['Body'].fillna('')
exploit_df['combined_text'] = (exploit_df['Subject'] + ' ' + exploit_df['Body']).str.strip()

# Calculate embeddings for the combined text
exploit_df['embeddings'] = exploit_df['combined_text'].apply(lambda x: model.encode(x))

# Function to perform cosine similarity search
def search_similar(query, top_n=3):
    query_embedding = model.encode(query).reshape(1, -1)
    embeddings = np.vstack(exploit_df['embeddings'].values)
    similarities = cosine_similarity(query_embedding, embeddings).flatten()
    top_indices = similarities.argsort()[-top_n:][::-1]
    results = exploit_df.iloc[top_indices][['Subject', 'Body', 'combined_text']].copy()
    results['similarity_score'] = similarities[top_indices]
    results['preview'] = results['combined_text'].apply(lambda x: x[:100] if x else 'No content available')
    return results[['preview', 'similarity_score']]

exploit_df.shape

c:\Users\Dave Sisk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


(34, 4)

In [4]:
exploit_df.sample(3)

,Subject,Body,combined_text,embeddings
31,Escrow Compliance Exception,"Move $102,000 as per the escrow directive. Leg...","Escrow Compliance Exception Move $102,000 as p...","[0.054614913, 0.02986592, -0.015277118, -0.040..."
8,Re: Final Settlement,"As discussed, please finalize the $150,000 pay...","Re: Final Settlement As discussed, please fina...","[-0.039154477, 0.07341119, 0.036629, -0.068680..."
16,RE: Reimbursement Urgency,"Due to an internal booking error, reimburse $4...",RE: Reimbursement Urgency Due to an internal b...,"[-0.025269803, 0.098000735, -0.02679673, 0.023..."


In [5]:
# Example query with a custom number of top matches
query = "urgent action required for account security"
top_n = 5  # Specify the number of top matches to return
results = search_similar(query, top_n=top_n)
print(results)

                                              preview  similarity_score
3   Urgent from Mobile I’m traveling and can’t acc...          0.497088
7   Immediate Compliance Wire I’ve been told we ha...          0.489520
22  Banking Delay Mitigation I’ve been notified th...          0.480011
13  Transfer Instruction: Urgent Initiate a transf...          0.461940
0   Urgent Wire Transfer – Confidential Hi [Employ...          0.421657


In practice, we would build this library via the reinforcement learning loop at the bottom right in the framework flowchart.  This means that we would start with little or no examples, and accumulate them as we proceeded, eventually amassing a large library of labeled data as examples. 

Also in practice, these vector search queries would be run against a vector database hydrated with the above-mentioned data instead of against a temporary in-memory dataframe.

## Pre-processing of Working Examples

Our first order of business is to get text transcriptions of the messages in these examples, so we can keep any vector search functionality in the text-only realm where we have known good models for that functionality. We'll leverage AI tooling to accomplish these transcriptions.

### Audio message
We can transcribe from the WAV audio file directly using the smallest open-source Vosk model that has an internalized language graph...this should produce a reasonably accurate transcription. (If the audio were in compressed MP3 file format [very likely if it came in an email, for instance], we'd have to convert it to WAV audio format first, then transcrible the text from that. That's merely a requirement of this particular model though, not the overall technology.)

We will use the Vosk models for transcription...these are small in size, free/open-source, and run locally, meaning no data leaves the current host in API calls.

In [ ]:
#! pip install vosk

In [6]:
# Let's start with the audio transcription
from vosk import Model, KaldiRecognizer
import wave
import json
import textwrap

# Load the 2nd smallest Vosk model
model = Model("./vosk-model-en-us-0.22-lgraph")

wav_file = './deepfake_DonaldTrump-audio.wav'

# Open the audio file
with wave.open(wav_file, "rb") as wf:
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() not in [8000, 16000]:
        raise ValueError("Audio file must be WAV format mono PCM.")
    
    recognizer = KaldiRecognizer(model, wf.getframerate())
    recognizer.SetWords(True)
    
    transcription = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if recognizer.AcceptWaveform(data):
            result = json.loads(recognizer.Result())
            transcription.append(result.get("text", ""))
    
    # Get the final transcription
    final_result = json.loads(recognizer.FinalResult())
    transcription.append(final_result.get("text", ""))

# Combine all parts of the transcription
transcribed_text_audio = " ".join(transcription)

# Wrap the text for better readability
wrapped_text_audio = textwrap.fill(transcribed_text_audio, width=60)
print("Transcribed Audio Text:\n", wrapped_text_audio)

Transcribed Audio Text:
 hi shane i'm following up on the audit readiness review
issue in q one expenses for contractor payments we need to
transfer one thirty seven thousand eight hundred twenty
dollar zero sense to the holding account a b c one twenty
three this must be sent by three pm today so do d o g e can
reflected in the pre-ordered submission let me know once the
transfer is done thanks


Next, we'll transcribe text from the audio track on the video to get the exploit message.  This requires multiple steps to get the audio track from the video, then transcribe text from the audio. Extracting audio from the video is easy and deterministic...typical tooling can be used for that task.  AI tech is again needed to transcribe the audio to text...we'll use the same Vosk models for the text transcription. 

In [7]:
# Let's transcribe text from the video
import os
import wave
from vosk import Model, KaldiRecognizer
import json
import textwrap

# Load the Vosk model
model = Model("./vosk-model-en-us-0.22-lgraph")

# Path to the video file
video_file = './deepfake_ElonMusk_SocialSecurityPhishing.mp4'
audio_file = './extracted_audio.wav'

# Delete the extracted audio file if it already exists
if os.path.exists(audio_file):
    os.remove(audio_file)

# Extract audio using ffmpeg (no moviepy)
!ffmpeg -i {video_file} -vn -acodec pcm_s16le -ar 16000 -ac 1 {audio_file}

# Transcribe the extracted audio
with wave.open(audio_file, "rb") as wf:
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getframerate() not in [8000, 16000]:
        raise ValueError("Audio file must be WAV format mono PCM.")
    
    recognizer = KaldiRecognizer(model, wf.getframerate())
    recognizer.SetWords(True)
    
    transcription = []
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if recognizer.AcceptWaveform(data):
            result = json.loads(recognizer.Result())
            transcription.append(result.get("text", ""))
    
    # Get the final transcription
    final_result = json.loads(recognizer.FinalResult())
    transcription.append(final_result.get("text", ""))

# Combine all parts of the transcription
transcribed_text_video = " ".join(transcription)

# Wrap the text for better readability
wrapped_text_video = textwrap.fill(transcribed_text_video, width=60)
print("\nTranscribed Video Text:\n", wrapped_text_video)

ffmpeg version 7.1.1-essentials_build-www.gyan.dev Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enable-libvpl --enable-nvdec --enable-nvenc --enable-vaapi --enable-libgme --enable-libopenmpt --enable-libopencore-amrwb --enable-libmp3lame -


Transcribed Video Text:
 greetings valued employees immediate action is needed from
you please go to the listed u r l logging into your social
security account and change your password this will provide
a immediate flag that your social security account is valid
please complete this task task by five pm today thank you


## Take these examples combined with typical non-attack communications, and run them through the proposed process.

In [8]:
import pandas as pd

# Load the CSV file into a pandas dataframe
incoming_df = pd.read_csv('./corporate_communication_examples.csv')

# Add the transcribed audio and video text as new rows
incoming_df = pd.concat([
    incoming_df,
    pd.DataFrame({'Subject': ['Action required: Transfer by 3pm todayTranscribed Audio'], 'Body': [transcribed_text_audio]}),
    pd.DataFrame({'Subject': ['Message to all employees'], 'Body': [transcribed_text_video]})
], ignore_index=True)


In [9]:

# Display the first few rows of the dataframe to verify the data
incoming_df.sample(3)

,Subject,Body
91,Team Building Activity,This is a friendly reminder to submit your tim...
34,Client Follow-up Required,Attached is the latest draft of the policy doc...
101,Message to all employees,greetings valued employees immediate action is...


In [10]:
incoming_df.tail(3)

,Subject,Body
99,Internal Survey Participation,Please confirm your attendance for next Thursd...
100,Action required: Transfer by 3pm todayTranscri...,hi shane i'm following up on the audit readine...
101,Message to all employees,greetings valued employees immediate action is...
